In [ ]:
import networkx as nx
import random
import os
import pickle
import tqdm
def generate_urban_network(grid_size=(3,3), randomness=0.2, filePath='map'):
    """
    Generates an urban-like road network based on a grid structure with random modifications.
    
    :param grid_size: Tuple (rows, columns) defining the base grid size.
    :param randomness: Probability of adding or removing edges to break uniformity.
    :param filePath: Path to save the SUMO network files.
    """
    G = nx.grid_2d_graph(grid_size[0], grid_size[1])  # Generate a base grid
    G = nx.convert_node_labels_to_integers(G)  # Convert to integer labels

    # Add random edges to simulate non-uniform urban planning
    nodes = list(G.nodes)
    num_extra_edges = int(randomness * len(nodes) * 2)  # More randomness for larger graphs

    for _ in range(num_extra_edges):
        u, v = random.sample(nodes, 2)
        if not G.has_edge(u, v):
            G.add_edge(u, v)

    # Ensure graph connectivity
    if not nx.is_connected(G):
        largest_component = max(nx.connected_components(G), key=len)
        isolated_nodes = set(G.nodes) - largest_component
        for node in isolated_nodes:
            nearest = min(largest_component, key=lambda x: nx.shortest_path_length(G, node, x))
            G.add_edge(node, nearest)

    # Convert to directed graph with bidirectional edges
    DG = nx.DiGraph()
    for u, v in G.edges():
        DG.add_edge(u, v)
        DG.add_edge(v, u)  # Ensure bidirectional roads

    # Generate SUMO network files
    networkx_to_sumo(DG, sumo_file=filePath + ".net.xml", node_file=filePath + ".nod.xml", edge_file=filePath + ".edg.xml")

    return DG

def networkx_to_sumo(G, sumo_file="myNetwork.net.xml", node_file='nodes.nod.xml', edge_file='edges.edg.xml'):
    scaling_factor = 700  # Scaling for SUMO visualization

    # Assign node positions in a pseudo-grid format
    pos = nx.spring_layout(G, scale=scaling_factor)  # Spring layout for visualization

    with open(node_file, 'w') as nodes:
        nodes.write('<nodes>\n')
        for node, (x, y) in pos.items():
            nodes.write(f'    <node id="{node}" x="{x}" y="{y}" type="priority"/>\n')
        nodes.write('</nodes>\n')

    with open(edge_file, 'w') as edges:
        edges.write('<edges>\n')
        for edge in G.edges():
            edges.write(f'    <edge id="{edge[0]}_{edge[1]}" from="{edge[0]}" to="{edge[1]}" priority="1"/>\n')
        edges.write('</edges>\n')

    # Ensure SUMO network conversion
    os.system(f'netconvert --node-files={node_file} --edge-files={edge_file} --output-file={sumo_file}')
    os.remove(node_file)
    os.remove(edge_file)

# Create a directory for all SUMO maps
output_dir = "sumomaps"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define grid sizes and randomness levels
# grid_sizes = [(n, n) for n in range(2, 11)]
# 
grid_sizes = [(n, n) for n in range(12, 51, 5)]  # From 2x2 to 10x10
randomness_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]  # Different randomness values

# Generate networks
for grid_size in tqdm.tqdm(grid_sizes):
    n = grid_size[0]  # Extract N value
    for randomness in randomness_levels:
        # for simEdition in range(50):
        for simEdition in range(2):
            p_str = f"p{int(randomness * 100):02d}"  # Convert randomness to pXX format
            folder_name = f"{output_dir}/n{n}_{p_str}_edition{str(simEdition)}"  # Create folder for this setting

            if not os.path.exists(folder_name):
                os.makedirs(folder_name)

            file_path = f"{folder_name}/map"
            G = generate_urban_network(grid_size=grid_size, randomness=randomness, filePath=file_path)

            with open(os.path.join(folder_name, "G"), 'wb') as f:
                pickle.dump(G, f)

print("✅ All networks generated and saved in 'sumomaps/' directory!")
